# Laboratorium 4
#### Elementy programowania funkcyjnego
Zaimplementuj poniższe funkcje tak, aby działały identycznie jak ich wbudowane opdowiedniki. Pamiętaj, że w Pythonie3 te funkcje zwracają generatory. W razie wątpliwości co do nietypowego zachowania tych funkcji odsyłam do <a href="https://docs.python.org/3/library/functions.html">dokumentacji</a>.

In [ ]:
def _filter(func=None, iterable=[]):
    """Filtruje z iterable elementy, dla których funkcja func zwraca False zostawiając pozostałe"""
    if func is None:
        func = lambda x: x
    for x in iterable:
        if func(x):
            yield x
            
from types import GeneratorType
print(isinstance(_filter(), GeneratorType))
print(list(filter(lambda x: x > 0, [0, -3, 1, 6])) == list(_filter(lambda x: x > 0, [0, -3, 1, 6])))
print(list(filter(None, [2, -3, 1, 6])) == list(_filter(None, [2, -3, 1, 6])))
print(list(filter(None, [True, False, False])) == list(_filter(None, [True, False, False])))
print(list(filter(None, [0, -3, 1, 6])) == list(_filter(None, [0, -3, 1, 6])))

In [ ]:
def _map(func, iterable, *args):
    """Mapuje elementy iterable na wartości fuknckji func"""
    for zipped_args in zip(iterable, *args):
        yield func(*zipped_args)


from types import GeneratorType
print(isinstance(_map(None, None), GeneratorType))
print(list(map(lambda x: x.upper(), 'ala ma kota')) == list(_map(lambda x: x.upper(), 'ala ma kota')))
print(list(map(lambda x,y: x+y, [1,2,3,4], [5,6,7,8])) == list(_map(lambda x,y: x+y, [1,2,3,4], [5,6,7,8])))

#### Jednolinijkowce
W poniższej serii zadań **nie** używaj list/dict/set comprehension, zamiast tego użyj paradygmatu funkcyjnego. Każde rozwiązanie powinno być jednolinijkowe.

In [ ]:
def celsius_to_fahrenheit(x):
    """Konwertuje liste temperatur w stopniach Celsjusza do skali Fahrenheita"""
    return map(lambda x: x * 1.8 + 32, x)

print(list(celsius_to_fahrenheit([0, 10, 100])) == [32.0, 50.0, 212.0])
print(list(celsius_to_fahrenheit([-123, 0])) == [-189.4, 32.0])

In [ ]:
from functools import reduce

def product_greater_than(x, k=0):
    """Zwraca iloczyn liczb w liście x większych od k"""
    return reduce(lambda x,y: x * y, filter(lambda x: x > k, x))

print(product_greater_than([1, 2, 3]) == 6)
print(product_greater_than([1, 2, 3], 2) == 3)
print(product_greater_than([-4, 5, 10, 23, 123], -5) == -565800)

In [ ]:
from functools import reduce

def create_sentence(x, k=0):
    """Łączy słowa (o długości co najmniej k) z listy x w zdanie"""
    return reduce(lambda x,y: x + ' ' + y, filter(lambda x: len(x) >= k, x))

print(create_sentence(['ala', 'ma', 'kota']) == 'ala ma kota')
print(create_sentence(['ala']) == 'ala')
print(create_sentence(['ala', 'ma', 'pieknego', 'kota'], k=3) == 'ala pieknego kota')

In [ ]:
def tuple_if_sum_greater(k, *lists):
    """Zwraca k-elementową krotke składającą się z kolejnych elementów list podanych jako arguemnty pozycyjne, 
       jeżeli ich suma jest większa niż parametr k"""
    return filter(lambda args: sum(args) > k, map(lambda *args: args, *lists))

print(list(tuple_if_sum_greater(0, [1,2,3])) == [(1,),(2,),(3,)])
print(list(tuple_if_sum_greater(4, [1,2,3], [2,3,4])) == [(2,3),(3,4)])
print(list(tuple_if_sum_greater(10, [1,2,3], [2,3,4])) == [])
print(list(tuple_if_sum_greater(0, [1,2], [3,4], [5,6])) == [(1,3,5), (2,4,6)])

In [ ]:
from math import sqrt

def primes(N):
    """Zwraca zbiór liczb pierwszych od 0 do N włącznie"""
    return set(filter(lambda i: all(map(lambda j: i % j, range(2, int((sqrt(i)))+1))), range(2, N+1)))

print(primes(5) == {2, 3, 5})
print(primes(10) == {2, 3, 5, 7})
print(primes(100) == {2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41,
                      43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97})

#### Dekoratory
Zaimplementuj poniższe dekoratory sprawdzające typy.

In [ ]:
def accepts(*types):
    """Sprawdza czy udekorowanej funckji zostały podane odpowiednie parametry zdefiniowane 
       w argumentach dekoratora"""
    def true_decorator(func):
        def wrapped_func(*args, **kwargs):
            all_args = args + tuple(kwargs.values())
            for arg, typ in zip(all_args, types):
                if not isinstance(arg, typ):
                    raise TypeError("Wrong type! Expected: {}, got: {}".format(typ, type(arg)))
            return func(*args, **kwargs)
        return wrapped_func
    return true_decorator


@accepts(str)
def capitalize(word):
    return word[0].upper() + word[1:]

print(capitalize('ola') == 'Ola')

try:
    capitalize(2)
except TypeError:
    print(True)

@accepts(float, int)
def static_pow(base, exp):
    return base ** exp 

print(static_pow(2., 2) == 4.)
print(static_pow(2., exp=2) == 4.)
print(static_pow(base=2., exp=2) == 4.)

try:
    static_pow('x', 10)
except TypeError:
    print(True)
    
try:
    static_pow(2, 2.2)
except TypeError:
    print(True)

In [ ]:
def returns(*types):
    """Sprawdza czy udekorowana funkcja zwraca poprawne argumenty, zdefiniowane w parametrach dekoratora"""
    def true_decorator(func):
        def wrapped_func(*args, **kwargs):
            returned = func(*args, **kwargs)
            for ret, typ in zip(returned, types):
                if not isinstance(ret, typ):
                    raise TypeError("Wrong type! Expected: {}, got: {}".format(typ, type(ret)))
            return returned
        return wrapped_func
    return true_decorator


@returns(str)
def str_only_identity(word):
    return word

print(str_only_identity('hello') == 'hello')

try:
    str_only_identity(10)
except TypeError:
    print(True)
    
@returns(int, int)
def split_indices(x):
    return x[0], x[1]

print(split_indices(x=[6,9]) == (6,9))

try:
    split('AB')
except TypeError:
    print(True)